In [1]:
import os

from google.cloud import bigquery
import pandas as pd

import log
import peloton_instructor
import peloton_user
import peloton_workout

USERNAME = os.environ['USERNAME']
PASSWORD = os.environ['PASSWORD']

# configure logging
logger = log.setup_custom_logger('peloton')

In [2]:
user = peloton_user.PelotonUser(USERNAME, PASSWORD)

# fetch list of workout ids
user.workout_ids = user.get_workout_ids()

INFO - peloton_user - Successfully logged in as xmarcosx
INFO - peloton_user - Returning 79 workout ids


In [3]:
# create workout objects
user.workouts = [peloton_workout.PelotonWorkout(user, workout_id) for workout_id in user.workout_ids]


In [4]:
output = {
    'name': [],
    'workout_id': [],
    'workout_datetime': [],
    'fitness_discipline': [],
    'title': []
}

for workout in user.workouts:
    output['name'].append(user.name)
    output['workout_id'].append(workout.workout_id)
    # convert seconds since epoch to datetime object
    output['workout_datetime'].append(pd.to_datetime(workout.created_at_epoch, unit='s'))
    # capitalize first letter of fitness discipline
    output['fitness_discipline'].append(workout.fitness_discipline.capitalize())
    output['title'].append(workout.ride_title)

df = pd.DataFrame(output)

df.tail(3)



,name,workout_id,workout_datetime,fitness_discipline,title
76,Marcos Alcozer,78a151a3a69941b6bfcedc603ee38c2d,2020-03-13 16:42:37,Cycling,45 min Groove Ride
77,Marcos Alcozer,47b3c8c8656f472e9a4e68fb9d477b89,2020-03-08 22:05:56,Running,20 min Intervals Run
78,Marcos Alcozer,9c0af5aa24724400bc67234c250d189b,2020-03-07 14:43:26,Cycling,30 min Groove Ride


In [9]:
client = bigquery.Client()

table_id = 'peloton.workouts'

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("name", "STRING"),
        bigquery.SchemaField("workout_id", "STRING"),
        bigquery.SchemaField("workout_datetime", "TIMESTAMP"),
        bigquery.SchemaField("fitness_discipline", "STRING"),
        bigquery.SchemaField("title", "STRING"),
    ],
    write_disposition="WRITE_TRUNCATE"
)

job = client.load_table_from_dataframe(df, table_id, job_config=job_config)

job.result()

In [6]:
# pull workout summary data
# [workout.get_workout_summary() for workout in workouts]

In [7]:
# pull performance graph data
# [workout.get_performance_graph() for workout in workouts]